# Reading Movies

In [335]:
import ast
import csv
score_scheme = []
with open('ScoreScheme.csv') as f:
    reader = csv.DictReader(f)
    lst = list(reader)
    for item in lst:
        if item["movie_name"] == "":
            continue
        if item["video_url"] == "None":
            continue
        score_scheme.append(item)

In [336]:
len(score_scheme)

2870

In [337]:
movies = []
with open('Output.csv') as f:
    reader = csv.DictReader(f)
    lst = list(reader)
    for item in lst:
        item["query_results"] = ast.literal_eval(item["query_results"])
        movies.append(item)

In [338]:
len(movies)

1967

# Merging Score Scheme File and Movies

In [339]:
movies_with_prerelease = []
for movie in movies:
    if movie["no_prerelease"] == "1":
        continue
    movies_with_prerelease.append(movie)
len(movies_with_prerelease)

1051

In [340]:
index_ignore = []
movies_with_results = []
for movie in movies_with_prerelease:
    movie["score_scheme_results"] = []
    found_movie = False
    for index, result in enumerate(score_scheme):
        if movie["movie_name"].replace(",", "") == result["movie_name"]:
            if index not in index_ignore:
                found_movie = True
                movie["score_scheme_results"].append(result)
                index_ignore.append(index)
        else:
            if found_movie:
                break
    movies_with_results.append(movie)

movies = movies_with_results

In [341]:
pending_to_select_trailers = []
new_movies = []
for movie in movies:
    if movie["score_scheme_results"] == []:
        pending_to_select_trailers.append(movie)
    else:
        new_movies.append(movie)
movies = new_movies
print("Pending to select ",len(pending_to_select_trailers))
print("New movies ",len(new_movies))

Pending to select  39
New movies  1012


In [342]:
ampliated_movies = []
for movie in movies:
    ampliated_results = []
    for result in movie["score_scheme_results"]:
        not_ampliated_video_id = result["video_url"].replace("https://www.youtube.com/watch?v=", "")
        found = False
        for query_result in movie["query_results"]:
            title, channel, video_id, result_date_str, premiered, stream, subscribers, views, likes = query_result
            if not_ampliated_video_id == video_id:
                result["release_date"] = result_date_str
                ampliated_results.append(result)
                found = True
                break
        if not found:
            print("Not found {}".format(result["video_url"]))
            result["release_date"] = "TO-COMPLETE"
            ampliated_results.append(result)
    movie["score_scheme_results"] = ampliated_results
    ampliated_movies.append(movie)
movies = ampliated_movies

Not found https://www.youtube.com/watch?v=W6xUnDKcHNA
Not found https://www.youtube.com/watch?v=0sH0__SpV88
Not found https://www.youtube.com/watch?v=HydiAICZq6I


In [343]:
completed_movies = []
for movie in movies:
    results = []
    for result in movie["score_scheme_results"]:
        if result["video_url"] == "https://www.youtube.com/watch?v=W6xUnDKcHNA":
            result["release_date"] = '7/23/2013'
        if result["video_url"] == "https://www.youtube.com/watch?v=0sH0__SpV88":
            result["release_date"] = '3/9/2015'
        if result["video_url"] == "https://www.youtube.com/watch?v=HydiAICZq6I":
            result["release_date"] = "6/3/2014"
        results.append(result)
    movie["score_scheme_results"] = results
    completed_movies.append(movie)
movies = completed_movies

In [344]:
from datetime import date

def days_diff(date1, date2):
    try:
        movie_moth, movie_day, movie_year = date2.split("/")
        result_moth, result_day, result_year = date1.split("/")
        movie_date = date(int(movie_year), int(movie_moth), int(movie_day))
        result_date = date(int(result_year), int(result_moth), int(result_day))
        days_diff_video_movie = (result_date - movie_date).days
    except Exception as e:
        days_diff_video_movie = 1
    return days_diff_video_movie

new_movies = []
for movie in movies:
    results = []
    for result in movie["score_scheme_results"]:
        result["days_diff"] = days_diff(result["release_date"], movie["release_start"])
        if result["days_diff"] < 0:
            results.append(result)
    movie["score_scheme_results"] = results
    new_movies.append(movie)
movies = new_movies

In [345]:
movies_with_selected_trailers = []
for movie in movies:
    results = []
    for result in movie["score_scheme_results"]:
        if result["official"] == "" and result["teaser"] == "":
            continue
        results.append(result)
    movie["selected_trailers"] = results
    movies_with_selected_trailers.append(movie)
movies = movies_with_selected_trailers

new_movies = []
for movie in movies:
    if movie["selected_trailers"] == []:
        pending_to_select_trailers.append(movie)
    else:
        new_movies.append(movie)
movies = new_movies
print("Pending to select ",len(pending_to_select_trailers))
print("New movies ",len(new_movies))

Pending to select  111
New movies  940


In [346]:
movies_that_dont_need_extra_attention = []
movies_that_need_extra_attention = []
for movie in movies:
    need_extra_attention = False
    for result in movie["score_scheme_results"]:
        if result["days_diff"] <= -365:
            need_extra_attention = True

    if need_extra_attention:
        movies_that_need_extra_attention.append(movie)
    else:
        movies_that_dont_need_extra_attention.append(movie)

In [347]:
movies_that_may_dont_need_scrape = []
movies_that_need_scrape = []
for movie in pending_to_select_trailers:
    if movie["score_scheme_results"] == []:
        movies_that_need_scrape.append(movie)
    else:
        movies_that_may_dont_need_scrape.append(movie)

In [348]:
with open("./new_score_scheme.csv", "w") as f:
    f.write(
        "uniqueID,movie_name,video_url,video_title,channel,score,id,official,teaser,days_diff,need_extra_attention,need_to_select_trailer")
    f.write("\r\n")
    for movie in movies_that_dont_need_extra_attention:
        if movie["score_scheme_results"] == []:
            print("Alho")
        movie_name = movie["movie_name"].replace(",", "")
        for result in movie["score_scheme_results"]:
            f.write("{},{},{},{},{},{},{},{},{},{},{},{}".format(
                movie["uniqueID"],
                movie_name,
                result["video_url"],
                result["video_title"],
                result["channel"],
                result["score"],
                result["id"],
                result["official"],
                result["teaser"],
                result["days_diff"],
                0,
                0
            ))
            f.write("\r\n")
        f.write(",,,,,,,,,,,")
        f.write("\r\n")

    for movie in movies_that_need_extra_attention:
        if movie["score_scheme_results"] == []:
            print("Alho")
        movie_name = movie["movie_name"].replace(",", "")
        for result in movie["score_scheme_results"]:
            f.write("{},{},{},{},{},{},{},{},{},{},{},{}".format(
                movie["uniqueID"],
                movie_name,
                result["video_url"],
                result["video_title"],
                result["channel"],
                result["score"],
                result["id"],
                result["official"],
                result["teaser"],
                result["days_diff"],
                1,
                0
            ))
            f.write("\r\n")
        f.write(",,,,,,,,,,,")
        f.write("\r\n")

In [349]:
"""
with open("./new_score_scheme.csv", "a") as f:
    for movie in movies_that_may_dont_need_scrape:
        movie_name = movie["movie_name"].replace(",", "")
        for result in movie["score_scheme_results"]:
            f.write("{},{},{},{},{},{},{},{},{},{},{},{}".format(
                movie["uniqueID"],
                movie_name,
                result["video_url"],
                result["video_title"],
                result["channel"],
                result["score"],
                result["id"],
                "",
                "",
                result["days_diff"],
                0,
                1
            ))
            f.write("\r\n")
        f.write(",,,,,,,,,,,")
        f.write("\r\n")

    for movie in movies_that_need_scrape:
        movie_name = movie["movie_name"].replace(",", "")
        f.write("{},{},{},{},{},{},{},{},{},{},{},{}".format(
                movie["uniqueID"],
                movie_name,
                None,
                None,
                None,
                None,
                None,
                "",
                "",
                None,
                0,
                1
            ))
        f.write("\r\n")
        f.write(",,,,,,,,,,,")
        f.write("\r\n")
"""

'\nwith open("./new_score_scheme.csv", "a") as f:\n    for movie in movies_that_may_dont_need_scrape:\n        movie_name = movie["movie_name"].replace(",", "")\n        for result in movie["score_scheme_results"]:\n            f.write("{},{},{},{},{},{},{},{},{},{},{},{}".format(\n                movie["uniqueID"],\n                movie_name,\n                result["video_url"],\n                result["video_title"],\n                result["channel"],\n                result["score"],\n                result["id"],\n                "",\n                "",\n                result["days_diff"],\n                0,\n                1\n            ))\n            f.write("\r\n")\n        f.write(",,,,,,,,,,,")\n        f.write("\r\n")\n\n    for movie in movies_that_need_scrape:\n        movie_name = movie["movie_name"].replace(",", "")\n        f.write("{},{},{},{},{},{},{},{},{},{},{},{}".format(\n                movie["uniqueID"],\n                movie_name,\n                None

In [350]:
exclude_channels = []
with open("Hand_Excluded_Channels.csv") as f:
  reader = csv.reader(f)
  lst = reader
  for item in lst:
    exclude_channels.append(item[0])

with open("Programmatically_Excluded_Channels.csv") as f:
  reader = csv.reader(f)
  lst = reader
  for item in lst:
    exclude_channels.append(item[0])


In [351]:
import re
def translate(to_translate):
		tabin = u'áéíóú'
		tabout = u'aeiou'
		tabin = [ord(char) for char in tabin]
		translate_table = dict(zip(tabin, tabout))
		return to_translate.translate(translate_table)

def remove_non_alphanumeric(result):
  return re.sub(r'[^a-zA-Z0-9]', '', result)

def normalize(token):
		result = token.lower()
		result = translate(result)
		result = remove_non_alphanumeric(result)
		return result

def score(title_tokens, channel_tokens, query_tokens):
  normalized_title_tokens = []
  for title_token in title_tokens:
    normalized_title_token = normalize(title_token)
    if len(normalized_title_token) > 1:
      normalized_title_tokens.append(normalized_title_token)

  normalized_channel_tokens = []
  for channel_token in channel_tokens:
    normalized_channel_tokens.append(normalize(channel_token))

  normalized_query_tokens = []
  for query_token in query_tokens:
    normalized_query_token = normalize(query_token)
    if len(normalized_query_token) > 1:
      normalized_query_tokens.append(normalized_query_token)

  matches = 0
  for normalized_query_token in normalized_query_tokens:
    if normalized_query_token in normalized_title_tokens:
      matches += 1
  score = matches / len(normalized_query_tokens)
  return score

In [352]:
with open("./new_score_scheme.csv", "a") as f:
    for movie in movies_that_may_dont_need_scrape:
        movie_name = movie["movie_name"].replace(",", "")

        candidates = []

        anterior_query_results = []
        for query_result in movie["query_results"]:
            title, channel, video_id, result_date_str, premiered, stream, subscribers, views, likes = query_result
            query_result.append(days_diff(result_date_str, movie["release_start"]))
            if days_diff(result_date_str, movie["release_start"]) < 0:
                anterior_query_results.append(query_result)

        candidates = anterior_query_results

        #exclude_channels_results = []
        #for anterior_query_result in anterior_query_results:
        #    title, channel, video_id, result_date_str, premiered, stream, subscribers, views, likes, days_diff_value = anterior_query_result
        #    if channel.lower() not in exclude_channels:
        #        exclude_channels_results.append(anterior_query_result)

        #candidates = exclude_channels_results
        #if exclude_channels_results == []:
        #    candidates = anterior_query_results

        if candidates == []:
            print("Error")

        for result in candidates:
            title, channel, video_id, result_date_str, premiered, stream, subscribers, views, likes, days_diff_value = result

            youtube_url = "https://www.youtube.com/watch?v={}".format(video_id)

            title = title.replace(",", "")
            channel = channel.replace(",", "")

            channel_tokens = channel.split(" ")
            title_tokens = title.split(" ")
            query_tokens = "{} official trailer".format(movie["movie_name"]).split()
            score_result = score(channel_tokens=channel_tokens, query_tokens=query_tokens, title_tokens=title_tokens)

            f.write("{},{},{},{},{},{},{},{},{},{},{},{}".format(
                movie["uniqueID"],
                movie_name,
                youtube_url,
                title,
                channel,
                score_result,
                0,
                "",
                "",
                days_diff_value,
                0,
                1
            ))
            f.write("\r\n")
        f.write(",,,,,,,,,,,")
        f.write("\r\n")


In [353]:
with open("./new_score_scheme.csv", "a") as f:
    for movie in movies_that_need_scrape:
        movie_name = movie["movie_name"].replace(",", "")

        candidates = []

        anterior_query_results = []
        for query_result in movie["query_results"]:
            title, channel, video_id, result_date_str, premiered, stream, subscribers, views, likes = query_result
            query_result.append(days_diff(result_date_str, movie["release_start"]))
            if days_diff(result_date_str, movie["release_start"]) < 0:
                anterior_query_results.append(query_result)

        candidates = anterior_query_results

        #exclude_channels_results = []
        #for anterior_query_result in anterior_query_results:
        #    title, channel, video_id, result_date_str, premiered, stream, subscribers, views, likes, days_diff_value = anterior_query_result
        #    if channel.lower() not in exclude_channels:
        #        exclude_channels_results.append(anterior_query_result)

        #candidates = exclude_channels_results
        #if exclude_channels_results == []:
        #    candidates = anterior_query_results

        if candidates == []:
            print("Error")

        for result in candidates:
            title, channel, video_id, result_date_str, premiered, stream, subscribers, views, likes, days_diff_value = result

            youtube_url = "https://www.youtube.com/watch?v={}".format(video_id)

            title = title.replace(",", "")
            channel = channel.replace(",", "")

            channel_tokens = channel.split(" ")
            title_tokens = title.split(" ")
            query_tokens = "{} official trailer".format(movie["movie_name"]).split()
            score_result = score(channel_tokens=channel_tokens, query_tokens=query_tokens, title_tokens=title_tokens)

            f.write("{},{},{},{},{},{},{},{},{},{},{},{}".format(
                movie["uniqueID"],
                movie_name,
                youtube_url,
                title,
                channel,
                score_result,
                0,
                "",
                "",
                days_diff_value,
                0,
                1
            ))
            f.write("\r\n")
        f.write(",,,,,,,,,,,")
        f.write("\r\n")
